<a href="https://colab.research.google.com/github/SODAx12/fake-news-detector/blob/main/Fake_News_Prediction_LogiREG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article.We considered only the determining words.
5. label:
           1: Fake news
           0: real News





Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Data Pre-processing

In [ ]:
# loading the dataset to a pandas DataFrame
news = pd.read_csv('/content/train.csv')
check=pd.read_csv('/content/test.csv')

FileNotFoundError: ignored

In [ ]:
news.shape

In [ ]:
check.shape

In [ ]:
# print the first 5 rows of the dataframe
news.head()

In [ ]:
check.head()

In [ ]:
# replacing the null values with empty string
news= news.fillna('')
news.isnull().sum()

In [ ]:
check=check.fillna('')
check.isnull().sum()

In [ ]:
import string
import re
stop_word = stopwords.words('english')
def cleaning_data(x):
    #lower the value
    x =x.lower()
    #converting in ascii value
    x = x.encode('ascii' , 'ignore').decode()
    #removing all the stopword
    x = ' '.join([word for word in x.split(' ') if word not in stop_word])
    #remove mentions
    x = re.sub('@\S+' , ' ' , x)
    #remove url
    x = re.sub('https*\S+' , ' ' , x)
    #remove hastag
    x= re.sub('#\S+' , ' ' , x)
    #remove ticks and next character
    x = re.sub("\'\w+" , ' ' , x)
    #remove punctuations
    x = re.sub('[%s]' % re.escape(string.punctuation) , ' ' , x)
    #remove number
    x = re.sub(r'\w+\d+\w*' ,'  ' , x)
    #remove the over spaces
    x = re.sub('\s{2,}' , '' , x)
    return x    

In [ ]:
news.title = news.title.apply(cleaning_data)
news.author = news.author.apply(cleaning_data)
news.text =  news.text.apply(cleaning_data)

In [ ]:
# merging the author name and news title
news['content'] = news['author']+' '+news['title']
news.drop(['title' , 'author'] , axis = 1 , inplace = True)
news.head()

In [ ]:
check.title = check.title.apply(cleaning_data)
check.author = check.author.apply(cleaning_data)
check.text =  check.text.apply(cleaning_data)

check['content'] = check['author']+' '+check['title']
check.drop(['title' , 'author'] , axis = 1 , inplace = True)
check.head()

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [ ]:
port_stem = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news['content'] = news['content'].apply(stemming)
check['content'] = check['content'].apply(stemming)
#news['text'] = news['text'].apply(stemming)
#check['text'] = check['text'].apply(stemming)

In [ ]:
Y = news['label'].values
news.drop(['label'] , axis =1 , inplace =True)
X=news.content.values

In [ ]:
z=check.content.values

In [ ]:
print(X)
print(Y)

In [ ]:
print(X.shape)
Y.shape

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)
z=vectorizer.transform(z)

In [ ]:
print(X)
print("-----")
print(z)

Splitting the dataset to training & test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,stratify=Y)

Training the Model: Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

Evaluation

accuracy score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Making a Predictive System

In [ ]:
prediction = model.predict(z)

In [ ]:
print(prediction)